In [59]:
import numpy as np
from scipy.stats import pearsonr
import os
from PIL import Image
from scipy.stats import pearsonr
import cv2
import matplotlib.pyplot as plt

In [60]:
def sacarDiferencia(carpeta, newCarpeta):
    # Crear la carpeta de imágenes de diferencia si no existe
    if not os.path.exists(newCarpeta):
        os.makedirs(newCarpeta)

    # Obtener la lista de archivos en la carpeta
    archivos = os.listdir(carpeta)

    # Iterar sobre los archivos
    for archivo in archivos:
        # Comprobar si es un archivo de imagen
        if archivo.endswith(".jpg") or archivo.endswith(".png"):
            # Ruta completa de la imagen
            ruta_imagen = os.path.join(carpeta, archivo)

            # Cargar la imagen
            imagen = cv2.imread(ruta_imagen, 0)
            imagen = cv2.normalize(imagen, None, 0, 255, cv2.NORM_MINMAX)

            # Eliminación de ruido con filtrado de media
            imagen = cv2.medianBlur(imagen, 5)

            # Mejora de la nitidez utilizando filtrado bilateral
            imagen = cv2.bilateralFilter(imagen, 9, 75, 75)

            # Obtener dimensiones de la imagen
            alto, ancho = imagen.shape

            # Dividir la imagen en mitades
            mitad_izquierda = imagen[:, :ancho // 2]
            mitad_derecha = imagen[:, ancho // 2:]

            # Aplicar efecto espejo vertical a la mitad derecha
            mitad_derecha_espejo = cv2.flip(mitad_derecha, 1)

            # Calcular la diferencia entre las mitades izquierda y derecha
            # diferencia = cv2.subtract(mitad_izquierda, mitad_derecha_espejo)
            diferencia = np.abs(mitad_izquierda.astype(int) - mitad_derecha_espejo.astype(int))

            # Convertir la imagen de diferencia a CV_8U
            diferencia = cv2.convertScaleAbs(diferencia)

            d = 3
            sigmaColor = 200
            sigmaSpace = 1000

            # Aplicar el filtro bilateral
            diferencia = cv2.bilateralFilter(diferencia, d, sigmaColor, sigmaSpace)

            # Guardar la imagen de diferencia en escala de grises
            nombre_grafica = "mitad_" + archivo
            ruta_grafica = os.path.join(newCarpeta, nombre_grafica)
            cv2.imwrite(ruta_grafica, diferencia)


In [61]:
import os
from PIL import Image
import numpy as np

def vectorizar(carpeta, label):
    archivos = os.listdir(carpeta)

    # Lista para almacenar los vectores de las imágenes
    vectores_imagenes = []

    # Iterar sobre los archivos
    for archivo in archivos:
        # Comprobar si es un archivo de imagen
        if archivo.endswith(".jpg") or archivo.endswith(".png"):
            # Ruta completa de la imagen
            ruta_imagen = os.path.join(carpeta, archivo)
            
            # Cargar la imagen
            imagen = Image.open(ruta_imagen)

            # Redimensiona la imagen a 8x8 píxeles
            imagen = imagen.resize((50, 125))

            # Convierte la imagen a escala de grises
            imagen = imagen.convert("L")

            # Convierte la imagen a un array de NumPy
            array_imagen = np.array(imagen)

            # Aplana el array de imagen a un vector de 1x64
            vector = array_imagen.flatten()
            
            # Crea una lista con la etiqueta y el nombre del archivo
            etiqueta_archivo = [label, archivo]

            # Agrega la lista al final del vector
            vector_con_etiqueta_archivo = np.append(vector, etiqueta_archivo)

            # Agrega el vector a la lista de vectores de imágenes
            vectores_imagenes.append(vector_con_etiqueta_archivo)

    # Convierte la lista de vectores en una matriz de NumPy
    return np.array(vectores_imagenes)


In [62]:
sacarDiferencia('yes_nuevo/','./carpeta_MitadesYes/')
data_yes = vectorizar('carpeta_MitadesYes/', 1)

sacarDiferencia('no_nuevo/','./carpeta_MitadesNo/')
data_no = vectorizar('carpeta_MitadesNo/', 0)


In [63]:
def divideTrainTest(data, porcentainTrain):
    # Número de filas a seleccionar
    num_filas_seleccionadas = int(porcentainTrain * data.shape[0])

    # Generar índices aleatorios de las filas a seleccionar
    indices_seleccionados = np.random.choice(data.shape[0], num_filas_seleccionadas, replace=False)

    # Crear la matriz con las filas seleccionadas
    Train = data[indices_seleccionados, :]

    # Crear la matriz con las filas no seleccionadas
    indices_no_seleccionados = np.setdiff1d(np.arange(data.shape[0]), indices_seleccionados)
    Test = data[indices_no_seleccionados, :]
    
    return Train, Test
    

In [64]:
def mergeData(data1, data2):
    return np.concatenate((data1, data2), axis=0)

In [65]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

class KNNClassifier:
    def __init__(self, k):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        y_pred = []
        for i in range(len(X_test)):
            distances = []
            for j in range(len(self.X_train)):
                dist = euclidean_distance(X_test[i], self.X_train[j])
                distances.append((dist, self.y_train[j]))
            distances.sort(key=lambda x: x[0])  # Ordenar distancias de menor a mayor
            neighbors = distances[:self.k]  # Obtener los k vecinos más cercanos
            classes = [neighbor[1] for neighbor in neighbors]  # Obtener las clases de los vecinos
            y_pred.append(max(set(classes), key=classes.count))  # Clasificación por voto mayoritario
        return y_pred
    
    

In [182]:
def knnAlgorithm(X_train, Y_train, X_test, Y_test, k):
    clases = np.unique(Y_train)
    confussionMatrix = np.zeros((len(clases),len(clases)), dtype=int)
    knn = KNNClassifier(k)
    knn.fit(X_train, Y_train)
    predictions = knn.predict(X_test)
    return predictions

In [67]:
class KMeans:
    def __init__(self, k, max_iterations):
        self.k = k
        self.max_iterations = max_iterations
        self.centroids = None
        self.labels = None

    def initialize_centroids(self, data):
        # Inicialización de centroides de manera aleatoria
        np.random.seed(0)
        indices = np.random.choice(data.shape[0], self.k, replace=False)
        centroids = data[indices]
        return centroids
    
    def initialize_centroids_with_features(self, data, real_labels):
        # Preprocesamiento de las imágenes
        flattened_images = data.reshape(data.shape[0], -1)  # Aplanar las imágenes en vectores de características
        unique_labels = np.unique(real_labels)

        # Cálculo de características promedio por etiqueta
        centroids = []
        for label in unique_labels:
            label_images = flattened_images[real_labels == label]
            label_mean = np.mean(label_images, axis=0)
            centroids.append(label_mean)

        # Selección de los primeros K centroides iniciales
        centroids = np.array(centroids)[:self.k]

        return centroids
    
    def kmeans_plusplus_initialization(self, data, k):
        centroids = []
        centroids.append(data[np.random.choice(data.shape[0])])  # Selecciona el primer centroide aleatoriamente

        for _ in range(1, k):
            distances = np.array([min([np.linalg.norm(point - centroid) for centroid in centroids]) for point in data])
            probabilities = distances / distances.sum()
            next_centroid_index = np.random.choice(data.shape[0], p=probabilities)
            centroids.append(data[next_centroid_index])

        return np.array(centroids)

    def assign_clusters(self, data):
        # Asignación de puntos a clústeres según la distancia euclidiana
        distances = np.sqrt(((data[:, np.newaxis] - self.centroids) ** 2).sum(axis=2))
        labels = np.argmin(distances, axis=1)
        return labels

    def update_centroids(self, data):
        # Actualización de los centroides como la media de los puntos asignados a cada clúster
        centroids = np.zeros((self.k, data.shape[1]))
        for i in range(self.k):
            cluster_data = data[self.labels == i]
            if len(cluster_data) > 0:
                centroids[i] = np.mean(cluster_data, axis=0)
        return centroids

    def fitRANDOM(self, data):
        self.centroids = self.initialize_centroids(data)

        for _ in range(self.max_iterations):
            prev_centroids = self.centroids.copy()

            self.labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)

            # Comprobar convergencia
            if np.all(prev_centroids == self.centroids):
                break
            
    def fitPreprocess(self, data, real_labels):
        self.centroids = self.initialize_centroids_with_features(data, real_labels)

        for _ in range(self.max_iterations):
            prev_centroids = self.centroids.copy()

            self.labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)

            # Comprobar convergencia
            if np.all(prev_centroids == self.centroids):
                break
    
    def fitKplus(self, data):
        self.centroids = self.initialize_centroids_with_features(data, self.k)

        for _ in range(self.max_iterations):
            prev_centroids = self.centroids.copy()

            self.labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)

            # Comprobar convergencia
            if np.array_equal(prev_centroids, self.centroids):
                break

    def predict(self, data):
        labels = self.assign_clusters(data)
        return labels

In [68]:
def calculate_confusion_matrix(true_labels, predicted_labels, num_classes):
    confusion_matrix = np.zeros((num_classes, num_classes))
    for true_label, predicted_label in zip(true_labels, predicted_labels):
        confusion_matrix[true_label][predicted_label] += 1
    return confusion_matrix

In [127]:
def compareKmeansAndKnn(X_train ,Y_train, X_test, Y_test, iterations, k = 2 ):
    kmeansPre = KMeans(k,iterations)
    kmeansR = KMeans(k,iterations)
    kmeansPlus = KMeans(k,iterations)
    # Ajustar el modelo a los datos
    kmeansPre.fitPreprocess(X_train, Y_train)
    kmeansR.fitRANDOM(X_train)
    kmeansPlus.fitKplus(X_train)
    # Obtener las etiquetas asignadas a cada punto
    predicted_labelsPre = kmeansPre.predict(X_test)
    predicted_labelsR = kmeansR.predict(X_test)
    predicted_labelsPlus = kmeansPlus.predict(X_test)
    # Calcular la matriz de confusión
    accuracyPre = np.mean(predicted_labelsPre == Y_test)
    accuracyR = np.mean(predicted_labelsPre == Y_test)
    accuracyPlus = np.mean(predicted_labelsPre == Y_test)
    
    '''confusion_matrixPre = calculate_confusion_matrix(Y_test, predicted_labelsPre, num_classes=k)
    confusion_matrixR = calculate_confusion_matrix(Y_test, predicted_labelsR, num_classes=k)
    confusion_matrixPlus = calculate_confusion_matrix(Y_test, predicted_labelsPlus, num_classes=k)
    
    # Imprimir la matriz de confusión
    print("Matriz de Confusión:")
    print(confusion_matrix)

    # Calcular la precisión como la suma de las coincidencias diagonales dividida por el número total de puntos
    accuracyPre = np.sum(np.max(confusion_matrixPre, axis=1)) / len(X_test)
    accuracyR = np.sum(np.max(confusion_matrixR, axis=1)) / len(X_test)
    accuracyPlus = np.sum(np.max(confusion_matrixPlus, axis=1)) / len(X_test)'''
    # Imprimir la precisión
    print("Precisión de clasificación con preprocesamiento: {:.2f}%".format(accuracyPre * 100))
    print("Precisión de clasificación con Random: {:.2f}%".format(accuracyR * 100))
    print("Precisión de clasificación con K++: {:.2f}% \n".format(accuracyPlus * 100))
    
    KNN = knnAlgorithm(X_train, Y_train, X_test, Y_test, 1)
    accuracyKNN = KNN[0]
    predicted_labelsKNN = KNN [1]
    # Crear una lista de tuplas con los valores de precisión y las etiquetas correspondientes
    resultados = [(accuracyPre, predicted_labelsPre ), (accuracyR, predicted_labelsR), (accuracyPlus, predicted_labelsPlus), (accuracyKNN, np.array(predicted_labelsKNN))]

    # Encontrar el resultado con la precisión más alta
    mejor_resultado = max(resultados, key=lambda x: x[0])

    # Devolver las etiquetas del mejor resultado
    return mejor_resultado[1]


In [118]:
data_dividedY = divideTrainTest(data_yes, 0.7)

data_trainY = data_dividedY[0]
data_testY = data_dividedY[1]


data_dividedN = divideTrainTest(data_no, 0.7)
data_trainN = data_dividedN[0]
data_testN = data_dividedN[1]
'''
print(data_yes.shape)
print(data_trainY.shape)
print(data_testY.shape)'''


'\nprint(data_yes.shape)\nprint(data_trainY.shape)\nprint(data_testY.shape)'

In [168]:
def exclude(matriz, indice):
    filas_seleccionadas = []
    for i, fila in enumerate(matriz):
        if i != indice:
            filas_seleccionadas.append(fila)
    return np.array(filas_seleccionadas)

In [71]:
dataTrain = mergeData(data_trainY, data_trainN)
print(dataTrain.shape) 
#print(dataTrain)
dataTest = mergeData(data_testY, data_testN)
print(dataTest.shape)
#print(dataTest)

(144, 6252)
(63, 6252)


In [88]:
ImgsTrain = np.array(dataTrain[:, :-2], dtype= int)
LabelsTrain = np.array(dataTrain[:, 6250], dtype= int)
NameImageTrain = np.array(dataTrain[:, 6251], dtype= str)

ImgsTest = np.array(dataTest[:, :-2], dtype= int)
LabelsTest = np.array(dataTest[:, 6250], dtype= int)
NameImageTest = np.array(dataTest[:, 6251], dtype= str)


In [152]:
def getBetterVersion(data_yes, data_no, porcentage):
    data_dividedY = divideTrainTest(data_yes, porcentage)
    data_trainY = data_dividedY[0]
    data_testY = data_dividedY[1]
    
    data_dividedN = divideTrainTest(data_no, porcentage)
    data_trainN = data_dividedN[0]
    data_testN = data_dividedN[1]
    
    
    dataTrain = mergeData(data_trainY, data_trainN)
    dataTest = mergeData(data_testY, data_testN)
    
    ImgsTrain = np.array(dataTrain[:, :-2], dtype= int)
    LabelsTrain = np.array(dataTrain[:, 6250], dtype= int)
    NameImageTrain = np.array(dataTrain[:, 6251], dtype= str)

    ImgsTest = np.array(dataTest[:, :-2], dtype= int)
    LabelsTest = np.array(dataTest[:, 6250], dtype= int)
    NameImageTest = np.array(dataTest[:, 6251], dtype= str)
    
    KNN = knnAlgorithm(ImgsTrain, LabelsTrain, ImgsTest, LabelsTest, 1)
    accuracyKNN = KNN[0]
    predicted_labelsKNN = np.array(KNN [1])
    print(predicted_labelsKNN.shape)
    newArchives = mergeData(NameImageTest, predicted_labelsKNN)
    newArchives = mergeData(newArchives, LabelsTest)
    newArchives = newArchives.reshape(3, newArchives.shape[0]//3) 
    print(newArchives.shape)
    return (newArchives)
    

In [153]:
archives = getBetterVersion(data_yes, data_no, 0.999)
import pandas as pd
df = pd.DataFrame({'Imagen': archives[0], 'Etiqueta': archives[1], 'Etiqueta real': archives[2]})
# Mostrar el DataFrame
print(df)

the accuracy with k =  1  is  100.0 % 
(2,)
(3, 2)
           Imagen Etiqueta Etiqueta real
0  mitad_Y128.jpg        1             1
1   mitad_N53.jpg        0             0


In [134]:
certeza = (df['Etiqueta'] == df['Etiqueta real']).sum() / len(df)

print(f"Nivel de certeza: {certeza}")

Nivel de certeza: 0.9090909090909091


In [190]:
import numpy as np
from sklearn.metrics import accuracy_score

def LOO_Knn(data_yes, data_no):
    allData = mergeData(data_yes, data_no)
    
    predicted_labels = []
    true_labels = []

    for i in range(allData.shape[0]):
        dataTrain = np.delete(allData, i, axis=0)  # Eliminar la fila correspondiente al índice i
        
        ImgsTrain = np.array(dataTrain[:, :-2], dtype=int)
        LabelsTrain = np.array(dataTrain[:, 6250], dtype=int)
        NameImageTrain = np.array(dataTrain[:, 6251], dtype=str)
        
        ImgsTest = np.array(allData[i, :-2], dtype=int)
        LabelsTest = np.array(allData[i, 6250], dtype=int)
        NameImageTest = np.array(allData[i, 6251], dtype=str)

        predicted_labelsKNN = np.array(knnAlgorithm(ImgsTrain, LabelsTrain, ImgsTest, LabelsTest, 1))
        print(predicted_labelsKNN.shape)
        print(i, predicted_labelsKNN, LabelsTest)
        predicted_labels.append(predicted_labelsKNN[0])  # Agregar solo el primer valor de la lista
        true_labels.append(LabelsTest)
        
    # Calcular la precisión general
    accuracy = accuracy_score(true_labels, predicted_labels)
    print("Precisión del clasificador KNN con Leave-One-Out: {:.2f}".format(accuracy))


In [191]:
LOO_Knn(data_yes, data_no)

(6250,)
0 [0 0 0 ... 0 0 0] 1


KeyboardInterrupt: 